# Pipe（管道）概念详解

理解进程间通信的核心机制

## 什么是 Pipe？

**Pipe = 进程之间的数据通道**（像水管一样传输数据）

```
┌─────────────┐     Pipe      ┌─────────────┐
│   父进程    │ ←──────────→ │   子进程    │
│  (Node.js)  │   双向数据流   │  (命令)     │
└─────────────┘              └─────────────┘
```

## 三种 Stdio 模式对比

| 模式 | 图示 | 数据流向 | 父进程能否捕获 |
|------|------|----------|---------------|
| `"inherit"` | 终端 ← 子进程 | 直接显示在屏幕 | ❌ 不能捕获 |
| `"pipe"` | 父进程 ↔ 子进程 | 通过管道传输 | ✅ 可以捕获 |
| `"ignore"` | × | 丢弃 | ❌ 没有数据 |

## 核心机制：事件驱动模型

### 事件发生的时刻

```javascript
// 时刻 1: 创建子进程（此时子进程开始独立运行）
const child = spawn("echo", ["Hello World"], {
  stdio: ["pipe", "pipe", "pipe"]
});

// 时刻 2: 注册事件监听器（此时子进程可能已经在运行了）
child.stdout.on("data", (data) => {
  console.log("✓ 捕获到:", data.toString());
});

child.on("close", (code) => {
  console.log("退出码:", code);
});

// 时刻 3: 当前脚本继续执行（如果有后续代码）
console.log("监听器注册完成");

// 时刻 4: 子进程输出数据 → 自动触发 "data" 事件
// 时刻 5: 子进程结束 → 自动触发 "close" 事件
```

### 事件触发机制（底层原理）

```
┌─────────────────────────────────────────────────────────┐
│                     Node.js 事件循环                      │
├─────────────────────────────────────────────────────────┤
│  1. spawn() 创建子进程（操作系统层面）                     │
│     ↓                                                   │
│  2. 子进程独立运行（并行执行命令）                         │
│     ↓                                                   │
│  3. 子进程输出数据到 stdout 管道                          │
│     ↓                                                   │
│  4. 操作系统通知 Node.js：管道有数据可读                   │
│     ↓                                                   │
│  5. Node.js 事件循环检测到 I/O 事件                       │
│     ↓                                                   │
│  6. 触发 "data" 事件，执行回调函数                        │
│     ↓                                                   │
│  7. 子进程结束，操作系统发送退出信号                       │
│     ↓                                                   │
│  8. 触发 "close" 事件                                    │
└─────────────────────────────────────────────────────────┘
```

### 可视化时间线

```
主进程（Node.js）          子进程（echo）
─────────────────          ─────────────────
spawn() 创建子进程  ───────► 开始执行 echo
注册事件监听器              输出 "Hello World"
继续执行后续代码            数据写入管道
                           ↓
检测到管道数据 ◄────────── 数据就绪
触发 "data" 事件
执行回调函数
                           进程结束
                           ↓
检测到子进程结束 ◄───────── 退出信号
触发 "close" 事件
执行回调函数
```

## 示例 1: inherit 模式（直接显示）

子进程输出直接显示在终端，父进程**无法捕获**

In [1]:
// Deno 使用 ES Modules 语法：import 而不是 require
import { spawn } from "node:child_process";

console.log("=== inherit 模式 ===");
console.log("子进程输出直接显示在终端\n");

const child = spawn("echo", ["Hello World"], {
  stdio: "inherit"  // 继承父进程的终端
});

// 尝试捕获输出（会失败，因为数据直接去了终端）
child.stdout?.on("data", (data) => {
  console.log("捕获到:", data.toString());  // 不会执行
});

child.on("close", (code) => {
  console.log(`\n子进程退出码: ${code}`);
  console.log("注意：Hello World 是子进程直接输出的");
});

// 防止 Deno 打印返回值
undefined;

=== inherit 模式 ===
子进程输出直接显示在终端


子进程退出码: 0
注意：Hello World 是子进程直接输出的


## 示例 2: pipe 模式（通过管道捕获）

子进程输出通过管道传输，父进程**可以捕获和处理**

In [2]:
import { spawn } from "node:child_process";

console.log("=== pipe 模式 ===");
console.log("子进程输出通过管道传输\n");

const child = spawn("echo", ["Hello World"], {
  stdio: ["pipe", "pipe", "pipe"]  // [stdin, stdout, stderr]
});

// 通过管道捕获子进程的 stdout
// 注意：这只是注册事件监听器，不会立即执行
child.stdout.on("data", (data) => {
  console.log("✓ 父进程捕获到子进程输出:");
  console.log(`  原始数据: ${data}`);
  console.log(`  处理后: ${data.toString().trim().toUpperCase()}`);
});

child.on("close", (code) => {
  console.log(`\n子进程退出码: ${code}`);
  console.log("注意：输出被父进程捕获并处理成大写了");
});

// 防止 Deno 打印返回值
undefined;

=== pipe 模式 ===
子进程输出通过管道传输

✓ 父进程捕获到子进程输出:
  原始数据: Hello World

  处理后: HELLO WORLD

子进程退出码: 0
注意：输出被父进程捕获并处理成大写了


### 关键理解：事件注册 vs 事件触发

| 代码 | 作用 | 执行时机 |
|------|------|----------|
| `child.stdout.on("data", callback)` | **注册**监听器 | 立即执行（同步） |
| `callback(data)` | **触发**回调 | 子进程有输出时（异步） |
| `child.on("close", callback)` | **注册**监听器 | 立即执行（同步） |
| `callback(code)` | **触发**回调 | 子进程结束时（异步） |

**类比**：就像订阅 YouTube 频道
- `subscribe()` = 注册监听器（立即完成）
- 新视频发布 = 事件发生（等待发生）
- 收到通知 = 回调函数执行（异步触发）

## 示例 3: 实际应用 - 处理命令输出

捕获 `ls -la` 命令的输出，只显示 JS 文件

In [7]:
import { spawn } from "node:child_process";

console.log("=== 实际应用：处理 ls -la 输出 ===\n");

const child = spawn("ls", ["-la"], {
  stdio: ["pipe", "pipe", "pipe"]
});

let output = "";

// 累积所有输出（每次子进程输出数据时触发）
child.stdout.on("data", (chunk) => {
  output += chunk.toString();
});

// 等子进程结束后再处理（子进程退出时触发）
child.on("close", (code) => {
  console.log(`子进程退出码: ${code}\n`);
  
  // 处理输出：只显示 .js 和 .mjs 文件
  const lines = output.split("\n");
  const jupyterFiles = lines.filter(line => line.endsWith(".ipynb"));
  
  console.log("找到的文件:");
  jupyterFiles.forEach(line => console.log(`  ${line}`));
  
  console.log(`\n总共 ${jupyterFiles.length} 个 Jupyter 文件`);
});

// 防止 Deno 打印返回值
undefined;

=== 实际应用：处理 ls -la 输出 ===

子进程退出码: 0

找到的文件:
  -rw-r--r--@  1 wangli  staff   9378 Feb  6 15:25 01-nodejs-basics.ipynb
  -rw-r--r--@  1 wangli  staff   8740 Feb  6 15:26 02-cli-commanderjs.ipynb
  -rw-r--r--@  1 wangli  staff  13427 Feb  6 15:36 03-pipe-concept.ipynb
  -rw-r--r--@  1 wangli  staff     72 Feb  6 15:23 Untitled.ipynb

总共 4 个 Jupyter 文件


## 什么时候用 pipe？

| 场景 | 使用 |
|------|------|
| 需要捕获子进程输出 | ✅ `stdio: ["pipe", "pipe", "pipe"]` |
| 需要处理/过滤子进程数据 | ✅ pipe |
| 只需要显示结果 | ❌ `stdio: "inherit"` 更简单 |

## OpenClaw 中的应用

在 `dist/entry.js` 中：

```javascript
const child = spawn(process.execPath, [...process.argv.slice(1)], {
  stdio: "inherit",  // 直接显示，不需要捕获
  env: process.env,
});
```

但如果需要捕获输出（比如执行命令后处理结果），就用 pipe。

## 总结

### Pipe 核心概念

```
┌─────────────────────────────────────────┐
│  Pipe = 进程之间的数据通道                │
│  就像水管连接两个水桶                    │
└─────────────────────────────────────────┘
```

### 事件驱动机制

- ✅ **spawn()** 创建子进程（立即返回，不等待）
- ✅ **.on()** 注册事件监听器（同步执行）
- ✅ **事件触发** 子进程产生输出/结束时自动触发（异步）
- ✅ **回调执行** Node.js 事件循环调用回调函数

### Stdio 模式

- ✅ **inherit**：直接显示，简单但无法捕获
- ✅ **pipe**：通过管道传输，可以捕获和处理
- ✅ **ignore**：丢弃数据